In [1]:
import requests
import scrapy
import re
REQ_HEADERS={'user-agent': 'UNC MEJO 583'}
resp = requests.get("http://goheels.com/roster.aspx?path=mbball", headers=REQ_HEADERS)
sel = scrapy.Selector(text=resp.text)

In [2]:
table = sel.css('table')[0]
headers = table.css('th').xpath('string()').extract()
players = []
for href in table.xpath('//tr//td//a/@href').extract():
    if 'roster.aspx' in href:
        res = requests.get("http://goheels.com"+href, headers=REQ_HEADERS)
        page = scrapy.Selector(text=res.text)
        info = page.css('.sidearm-roster-player-fields')
        headers = info.css('dt').xpath('string()').extract()
        data = info.css('dd').xpath('string()').extract()
        bio = dict(zip(headers, data))
        names = page.css('.sidearm-roster-player-name span::text').extract()
        bio['First Name'] = names[0]
        bio['Last Name'] = names[1]
        bio['Name'] = ' '.join(names)
        bio['Number'] = page.css('.sidearm-roster-player-jersey-number::text').extract_first()
        bio['Image'] = page.css('.sidearm-roster-player-image img::attr(src)').extract_first()
        player_id = re.search('rp_id=(\d+)', href).group(1)
        stats_response = requests.get("http://goheels.com/services/responsive-roster-bio.ashx?type=stats&rp_id=13515&path=mbball&year=2017&player_id="+player_id, headers=REQ_HEADERS).json()
        current_stats = scrapy.Selector(text=stats_response['current_stats'])
        stats_headers = current_stats.css('th').xpath('string()').extract()
        stats_numbers = current_stats.css('tfoot td').xpath('string()').extract()
        bio['Stats'] = dict(zip(stats_headers, stats_numbers))
        del bio['Stats']['Opponent']
        del bio['Stats']['GS']
        del bio['Stats']['Date']
        players.append(bio)

In [3]:
players

[{'Class': 'Sophomore',
  'First Name': 'Seventh',
  'Height': '6-1',
  'High School': 'Hammond School',
  'Hometown': 'Columbia, S.C.',
  'Image': '/images/2017/9/12/Woods_Seventh_mb_2017_181.jpg?width=300',
  'Last Name': 'Woods',
  'Name': 'Seventh Woods',
  'Number': '0',
  'Position': 'Guard',
  'Stats': {'%': '.810',
   '3FG/A': '5-34',
   'AST': '105',
   'AVG': '8.8',
   'BLK': '14',
   'DEF': '114',
   'FGM/A': '69-158',
   'FTM/A': '68-84',
   'MIN': '677',
   'OFF': '32',
   'PF': '48',
   'PTS': '211',
   'STL': '19',
   'T/O': '53',
   'TOT': '146'},
  'Weight': '185'},
 {'Class': 'Senior',
  'First Name': 'Theo',
  'Height': '6-6',
  'High School': 'Wesleyan Christian Academy',
  'Hometown': 'Greensboro, N.C.',
  'Image': '/images/2017/9/12/Pinson_Theo_mb_2017_23.jpg?width=300',
  'Last Name': 'Pinson',
  'Name': 'Theo Pinson',
  'Number': '1',
  'Position': 'Forward/Guard',
  'Stats': {'%': '.810',
   '3FG/A': '5-34',
   'AST': '105',
   'AVG': '8.8',
   'BLK': '14',
   